In [12]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import lightgbm as lgb
import catboost as ctb
import datetime as dt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
from sklearn.impute import SimpleImputer
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

In [13]:
borrar = ["Opportunity_Name", "ID", "Brand", "Product_Type", "ASP_converted_Currency", 
          "Prod_Category_A", "Product_Category_B", "Actual_Delivery_Date", "Last_Activity", 
          "ASP_converted_Currency", "Prod_Category_A", "Product_Name", "Delivery_Year", "Month", 
          "TRF", "Product_Family", "Account_Name"]

useless = ["Submitted_for_Approval", "Account_Type", "Delivery_Terms", "Size", "Price", "ASP_Currency", 
           "Total_Amount_Currency", "Total_Taxable_Amount_Currency","Quote_Type", "Opportunity_Type"] # "Product_Category_B", "Region"]

dates = ["Account_Created_Date", "Opportunity_Created_Date", "Quote_Expiry_Date", "Last_Modified_Date", 
         "Planned_Delivery_Start_Date", "Planned_Delivery_End_Date"]

target = ["Opportunity_ID", "Stage", "Sales_Contract_No"]

In [14]:
def preprocess(data):
    
    # elimino caracteres prohibidos en los headers
    
    data = data.rename(columns = lambda x:re.sub("[^A-Za-z0-9_]+", "", x))
        
    for d in dates:
        data[d] = pd.to_datetime(data[d])

    # agrego features
    
    data["Contacts"] = data.groupby("Opportunity_ID", sort = False)["Opportunity_ID"].transform("count")
    data["Delivery_Difference"] = (data["Planned_Delivery_End_Date"] - data["Planned_Delivery_Start_Date"]).dt.days
    data["Same_Owner"] = (data.Opportunity_Owner == data.Account_Owner) & (data.Opportunity_Owner == data.Last_Modified_By)
    data["Has_Brand"] = data.Brand != "None"
    data["Has_Contract"] = data.Sales_Contract_No != "None"
    data["Different_Country"] = (data.Billing_Country != data.Territory) & (data.Territory != "None")    
    data.loc[data.TRF == 0, "TRF_Cat"] = 0
    data.loc[(1 <= data.TRF) & (data.TRF <= 7), "TRF_Cat"] = 1
    data.loc[data.TRF > 7, "TRF_Cat"] = 2
    data["Sales"] = data.groupby("Account_Name", sort = False)["Account_Name"].transform("count")
    
    data["Concrete_Offer"] = (data["Planned_Delivery_End_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    data["Offer_Duration"] = (data["Quote_Expiry_Date"] - data["Opportunity_Created_Date"]).dt.days
    
    # fabri
    
    data["Territory_Defined"] = data.Territory != "None"
    data["Past_Quote"] = (data["Last_Modified_Date"] - data["Quote_Expiry_Date"]).dt.days
    
    # casteo a categoricas varias columnas
    categorical = [x for x in data.columns if data[x].dtype == "object"]
    for c in categorical:       
        data[c] = data[c].astype('category')
        
    # limpio columnas
    
    data = data.drop(borrar + useless + dates, axis = 1)
    
    if 'Stage' in data:
        data = data[(data.Stage == "Closed Won") | (data.Stage == "Closed Lost")]
        data.Stage = data.Stage.replace({"Closed Won": 1, "Closed Lost": 0})
    
    # numeric encode de categoricals
    
    label_encoder = LabelEncoder()
    cat_vars = [x for x in data.select_dtypes('category').columns if x != 'Stage']
    for col in cat_vars:
        data[col] = label_encoder.fit_transform(data[col])
        
    # elimino nans. strategy --> mean
    imputer = SimpleImputer(strategy='mean')
    cols = data.columns
    data = imputer.fit_transform(data)
    data = pd.DataFrame(data, columns = cols)
    
    return (data)

In [15]:
data = pd.read_csv("Entrenamieto_ECI_2020.csv")

In [16]:
prep = preprocess(data)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(prep, prep.Stage, test_size=0.3, random_state=0)

In [18]:
clf = ctb.CatBoostClassifier(num_boost_round=5000,
                               learning_rate=0.015,
                               l2_leaf_reg=3.5,
                               depth=8, 
                               max_bin=400,
                               num_leaves=31,
                               rsm=0.98,
                               eval_metric='Logloss',
                               random_seed=42)

In [19]:
model = AdaBoostClassifier(base_estimator=clf, random_state=42, n_estimators=5)

In [ ]:
model.fit(X=x_train.drop(target, axis=1), y=y_train)

0:	learn: 0.6657841	total: 83.1ms	remaining: 6m 55s
1:	learn: 0.6388557	total: 95.6ms	remaining: 3m 58s
2:	learn: 0.6127958	total: 105ms	remaining: 2m 54s
3:	learn: 0.5895563	total: 114ms	remaining: 2m 21s
4:	learn: 0.5658073	total: 122ms	remaining: 2m 2s
5:	learn: 0.5448163	total: 131ms	remaining: 1m 48s
6:	learn: 0.5231629	total: 141ms	remaining: 1m 40s
7:	learn: 0.5044360	total: 150ms	remaining: 1m 33s
8:	learn: 0.4848185	total: 158ms	remaining: 1m 27s
9:	learn: 0.4665669	total: 167ms	remaining: 1m 23s
10:	learn: 0.4507252	total: 175ms	remaining: 1m 19s
11:	learn: 0.4348217	total: 182ms	remaining: 1m 15s
12:	learn: 0.4207599	total: 191ms	remaining: 1m 13s
13:	learn: 0.4064184	total: 199ms	remaining: 1m 10s
14:	learn: 0.3922377	total: 207ms	remaining: 1m 8s
15:	learn: 0.3789233	total: 216ms	remaining: 1m 7s
16:	learn: 0.3661382	total: 226ms	remaining: 1m 6s
17:	learn: 0.3534983	total: 235ms	remaining: 1m 5s
18:	learn: 0.3410468	total: 244ms	remaining: 1m 3s
19:	learn: 0.3301206	total

169:	learn: 0.0799430	total: 1.71s	remaining: 48.7s
170:	learn: 0.0797691	total: 1.73s	remaining: 48.8s
171:	learn: 0.0796032	total: 1.74s	remaining: 48.8s
172:	learn: 0.0793994	total: 1.75s	remaining: 48.7s
173:	learn: 0.0791995	total: 1.75s	remaining: 48.7s
174:	learn: 0.0789903	total: 1.76s	remaining: 48.6s
175:	learn: 0.0788409	total: 1.77s	remaining: 48.6s
176:	learn: 0.0787351	total: 1.78s	remaining: 48.6s
177:	learn: 0.0784353	total: 1.79s	remaining: 48.5s
178:	learn: 0.0782886	total: 1.8s	remaining: 48.5s
179:	learn: 0.0781368	total: 1.81s	remaining: 48.5s
180:	learn: 0.0780120	total: 1.82s	remaining: 48.4s
181:	learn: 0.0777343	total: 1.83s	remaining: 48.4s
182:	learn: 0.0775144	total: 1.84s	remaining: 48.4s
183:	learn: 0.0774167	total: 1.84s	remaining: 48.3s
184:	learn: 0.0772388	total: 1.85s	remaining: 48.3s
185:	learn: 0.0771102	total: 1.86s	remaining: 48.2s
186:	learn: 0.0769627	total: 1.87s	remaining: 48.2s
187:	learn: 0.0767534	total: 1.88s	remaining: 48.2s
188:	learn: 0

348:	learn: 0.0576175	total: 3.34s	remaining: 44.5s
349:	learn: 0.0575516	total: 3.35s	remaining: 44.5s
350:	learn: 0.0574683	total: 3.36s	remaining: 44.5s
351:	learn: 0.0573541	total: 3.37s	remaining: 44.5s
352:	learn: 0.0572948	total: 3.38s	remaining: 44.5s
353:	learn: 0.0571804	total: 3.39s	remaining: 44.5s
354:	learn: 0.0571120	total: 3.4s	remaining: 44.5s
355:	learn: 0.0570433	total: 3.41s	remaining: 44.5s
356:	learn: 0.0569837	total: 3.42s	remaining: 44.5s
357:	learn: 0.0568981	total: 3.43s	remaining: 44.5s
358:	learn: 0.0568132	total: 3.44s	remaining: 44.5s
359:	learn: 0.0567082	total: 3.45s	remaining: 44.5s
360:	learn: 0.0565812	total: 3.46s	remaining: 44.5s
361:	learn: 0.0564495	total: 3.47s	remaining: 44.5s
362:	learn: 0.0563865	total: 3.48s	remaining: 44.5s
363:	learn: 0.0563003	total: 3.49s	remaining: 44.5s
364:	learn: 0.0562530	total: 3.5s	remaining: 44.5s
365:	learn: 0.0562072	total: 3.51s	remaining: 44.5s
366:	learn: 0.0561008	total: 3.52s	remaining: 44.5s
367:	learn: 0.

508:	learn: 0.0461873	total: 4.96s	remaining: 43.8s
509:	learn: 0.0460980	total: 4.97s	remaining: 43.8s
510:	learn: 0.0460318	total: 4.98s	remaining: 43.8s
511:	learn: 0.0459462	total: 5s	remaining: 43.8s
512:	learn: 0.0459211	total: 5.01s	remaining: 43.8s
513:	learn: 0.0459091	total: 5.01s	remaining: 43.8s
514:	learn: 0.0458184	total: 5.03s	remaining: 43.8s
515:	learn: 0.0457412	total: 5.04s	remaining: 43.8s
516:	learn: 0.0456705	total: 5.04s	remaining: 43.7s
517:	learn: 0.0455780	total: 5.05s	remaining: 43.7s
518:	learn: 0.0455332	total: 5.06s	remaining: 43.7s
519:	learn: 0.0454702	total: 5.07s	remaining: 43.7s
520:	learn: 0.0454136	total: 5.08s	remaining: 43.7s
521:	learn: 0.0453690	total: 5.09s	remaining: 43.6s
522:	learn: 0.0453070	total: 5.09s	remaining: 43.6s
523:	learn: 0.0452482	total: 5.1s	remaining: 43.6s
524:	learn: 0.0451443	total: 5.11s	remaining: 43.6s
525:	learn: 0.0450567	total: 5.12s	remaining: 43.6s
526:	learn: 0.0450323	total: 5.13s	remaining: 43.5s
527:	learn: 0.04

683:	learn: 0.0367572	total: 6.8s	remaining: 42.9s
684:	learn: 0.0367292	total: 6.82s	remaining: 43s
685:	learn: 0.0366535	total: 6.83s	remaining: 43s
686:	learn: 0.0366286	total: 6.84s	remaining: 42.9s
687:	learn: 0.0365761	total: 6.85s	remaining: 42.9s
688:	learn: 0.0365591	total: 6.86s	remaining: 42.9s
689:	learn: 0.0365352	total: 6.87s	remaining: 42.9s
690:	learn: 0.0364989	total: 6.88s	remaining: 42.9s
691:	learn: 0.0364603	total: 6.89s	remaining: 42.9s
692:	learn: 0.0363788	total: 6.89s	remaining: 42.9s
693:	learn: 0.0363727	total: 6.9s	remaining: 42.8s
694:	learn: 0.0363525	total: 6.91s	remaining: 42.8s
695:	learn: 0.0363384	total: 6.92s	remaining: 42.8s
696:	learn: 0.0363006	total: 6.93s	remaining: 42.8s
697:	learn: 0.0362499	total: 6.93s	remaining: 42.7s
698:	learn: 0.0361965	total: 6.94s	remaining: 42.7s
699:	learn: 0.0361259	total: 6.95s	remaining: 42.7s
700:	learn: 0.0360821	total: 6.96s	remaining: 42.7s
701:	learn: 0.0360285	total: 6.97s	remaining: 42.7s
702:	learn: 0.0359

...

1845:	learn: 0.4154930	total: 17.6s	remaining: 30s
1846:	learn: 0.4154782	total: 17.6s	remaining: 30s
1847:	learn: 0.4154664	total: 17.6s	remaining: 30s
1848:	learn: 0.4154543	total: 17.6s	remaining: 30s
1849:	learn: 0.4154432	total: 17.6s	remaining: 30s
1850:	learn: 0.4154270	total: 17.6s	remaining: 29.9s
1851:	learn: 0.4154095	total: 17.6s	remaining: 29.9s
1852:	learn: 0.4153867	total: 17.6s	remaining: 29.9s
1853:	learn: 0.4153767	total: 17.6s	remaining: 29.9s
1854:	learn: 0.4153655	total: 17.6s	remaining: 29.9s
1855:	learn: 0.4153525	total: 17.6s	remaining: 29.9s
1856:	learn: 0.4153483	total: 17.7s	remaining: 29.9s
1857:	learn: 0.4153356	total: 17.7s	remaining: 29.9s
1858:	learn: 0.4153301	total: 17.7s	remaining: 29.9s
1859:	learn: 0.4153226	total: 17.7s	remaining: 29.8s
1860:	learn: 0.4153041	total: 17.7s	remaining: 29.8s
1861:	learn: 0.4152869	total: 17.7s	remaining: 29.8s
1862:	learn: 0.4152700	total: 17.7s	remaining: 29.8s
1863:	learn: 0.4152599	total: 17.7s	remaining: 29.8s
186

2018:	learn: 0.4135337	total: 19.2s	remaining: 28.3s
2019:	learn: 0.4135258	total: 19.2s	remaining: 28.3s
2020:	learn: 0.4135210	total: 19.2s	remaining: 28.3s
2021:	learn: 0.4135133	total: 19.2s	remaining: 28.3s
2022:	learn: 0.4135041	total: 19.2s	remaining: 28.2s
2023:	learn: 0.4134962	total: 19.2s	remaining: 28.2s
2024:	learn: 0.4134825	total: 19.2s	remaining: 28.2s
2025:	learn: 0.4134763	total: 19.2s	remaining: 28.2s
2026:	learn: 0.4134646	total: 19.2s	remaining: 28.2s
2027:	learn: 0.4134580	total: 19.2s	remaining: 28.2s
2028:	learn: 0.4134537	total: 19.3s	remaining: 28.2s
2029:	learn: 0.4134481	total: 19.3s	remaining: 28.2s
2030:	learn: 0.4134403	total: 19.3s	remaining: 28.2s
2031:	learn: 0.4134315	total: 19.3s	remaining: 28.2s
2032:	learn: 0.4134218	total: 19.3s	remaining: 28.2s
2033:	learn: 0.4134179	total: 19.3s	remaining: 28.2s
2034:	learn: 0.4134117	total: 19.3s	remaining: 28.2s
2035:	learn: 0.4134049	total: 19.3s	remaining: 28.2s
2036:	learn: 0.4133940	total: 19.4s	remaining:

2184:	learn: 0.4121791	total: 20.8s	remaining: 26.8s
2185:	learn: 0.4121738	total: 20.8s	remaining: 26.8s
2186:	learn: 0.4121626	total: 20.8s	remaining: 26.8s
2187:	learn: 0.4121561	total: 20.8s	remaining: 26.8s
2188:	learn: 0.4121518	total: 20.8s	remaining: 26.8s
2189:	learn: 0.4121459	total: 20.8s	remaining: 26.7s
2190:	learn: 0.4121417	total: 20.8s	remaining: 26.7s
2191:	learn: 0.4121329	total: 20.9s	remaining: 26.7s
2192:	learn: 0.4121262	total: 20.9s	remaining: 26.7s
2193:	learn: 0.4121207	total: 20.9s	remaining: 26.7s
2194:	learn: 0.4121105	total: 20.9s	remaining: 26.7s
2195:	learn: 0.4121060	total: 20.9s	remaining: 26.7s
2196:	learn: 0.4120930	total: 20.9s	remaining: 26.7s
2197:	learn: 0.4120874	total: 20.9s	remaining: 26.7s
2198:	learn: 0.4120813	total: 20.9s	remaining: 26.6s
2199:	learn: 0.4120750	total: 20.9s	remaining: 26.6s
2200:	learn: 0.4120617	total: 20.9s	remaining: 26.6s
2201:	learn: 0.4120526	total: 20.9s	remaining: 26.6s
2202:	learn: 0.4120459	total: 21s	remaining: 2

2344:	learn: 0.4111262	total: 22.2s	remaining: 25.1s
2345:	learn: 0.4111203	total: 22.2s	remaining: 25.1s
2346:	learn: 0.4111124	total: 22.2s	remaining: 25.1s
2347:	learn: 0.4111067	total: 22.2s	remaining: 25.1s
2348:	learn: 0.4111014	total: 22.2s	remaining: 25.1s
2349:	learn: 0.4110941	total: 22.3s	remaining: 25.1s
2350:	learn: 0.4110882	total: 22.3s	remaining: 25.1s
2351:	learn: 0.4110798	total: 22.3s	remaining: 25.1s
2352:	learn: 0.4110748	total: 22.3s	remaining: 25.1s
2353:	learn: 0.4110695	total: 22.3s	remaining: 25.1s
2354:	learn: 0.4110614	total: 22.3s	remaining: 25s
2355:	learn: 0.4110568	total: 22.3s	remaining: 25s
2356:	learn: 0.4110524	total: 22.3s	remaining: 25s
2357:	learn: 0.4110476	total: 22.3s	remaining: 25s
2358:	learn: 0.4110432	total: 22.3s	remaining: 25s
2359:	learn: 0.4110301	total: 22.3s	remaining: 25s
2360:	learn: 0.4110225	total: 22.4s	remaining: 25s
2361:	learn: 0.4110164	total: 22.4s	remaining: 25s
2362:	learn: 0.4110125	total: 22.4s	remaining: 25s
2363:	learn

2517:	learn: 0.4101813	total: 23.8s	remaining: 23.5s
2518:	learn: 0.4101746	total: 23.8s	remaining: 23.5s
2519:	learn: 0.4101695	total: 23.9s	remaining: 23.5s
2520:	learn: 0.4101662	total: 23.9s	remaining: 23.5s
2521:	learn: 0.4101610	total: 23.9s	remaining: 23.4s
2522:	learn: 0.4101559	total: 23.9s	remaining: 23.4s
2523:	learn: 0.4101489	total: 23.9s	remaining: 23.4s
2524:	learn: 0.4101454	total: 23.9s	remaining: 23.4s
2525:	learn: 0.4101386	total: 23.9s	remaining: 23.4s
2526:	learn: 0.4101340	total: 23.9s	remaining: 23.4s
2527:	learn: 0.4101283	total: 23.9s	remaining: 23.4s
2528:	learn: 0.4101226	total: 23.9s	remaining: 23.4s
2529:	learn: 0.4101195	total: 23.9s	remaining: 23.4s
2530:	learn: 0.4101138	total: 23.9s	remaining: 23.4s
2531:	learn: 0.4101095	total: 24s	remaining: 23.3s
2532:	learn: 0.4101041	total: 24s	remaining: 23.3s
2533:	learn: 0.4101004	total: 24s	remaining: 23.3s
2534:	learn: 0.4100941	total: 24s	remaining: 23.3s
2535:	learn: 0.4100896	total: 24s	remaining: 23.3s
253

In [122]:
y_pred = model.predict_proba(x_test.drop(target, axis=1))

In [123]:
res = y_pred[:,1]

In [124]:
for x in res[:10]:
    print("{:f}".format(x))

0.000542
0.006752
0.999622
0.995506
0.015602
0.983827
0.003398
0.000147
0.995877
0.002002


In [68]:
x_test.Stage.head(10)

2040     0.0
4069     0.0
4230     1.0
578      1.0
12594    0.0
6042     1.0
2763     0.0
15900    0.0
2669     1.0
12802    0.0
Name: Stage, dtype: float64

In [125]:
fuck = [x for x in res if x >= 0.025 and x <= 0.95]
len(fuck)

165

In [126]:
len(fuck)/float(len(res)) * 100

3.257650542941757

In [127]:
pred = pd.DataFrame(res, index = x_test.index, columns = ["Prediction"])

In [128]:
x_test_pred = x_test.join(pred)
answer = x_test_pred.groupby("Opportunity_ID")["Stage","Prediction"].mean()

In [129]:
log_loss(answer["Stage"], answer["Prediction"])

0.07168224376072964

In [27]:
bad = x_test_pred[(x_test_pred.Prediction > 0.5) & (x_test_pred.Stage == 0) | (x_test_pred.Prediction < 0.5) & (x_test_pred.Stage == 1)]

In [29]:
bad.Stage.value_counts()

0    78
1    21
Name: Stage, dtype: int64

In [133]:
validation_file = "Validacion_ECI_2020.csv"
vali = pd.read_csv(validation_file)
validation = preprocess(vali)
    
leak = ["Opportunity_ID", "Sales_Contract_No"]
pred = model.predict_proba(validation.drop(leak, axis = 1))[:,1]

# agrupo por Opportunity_ID para dar una sola prediccion por solicitud

pred = pd.DataFrame(pred, index = validation.index, columns = ["Prediction"])
validation = validation.join(pred)

answer = pd.DataFrame(validation.groupby("Opportunity_ID", as_index = False)["Prediction"].mean())

In [134]:
prev = pd.read_csv("acceptable/submission_23.csv", names=["Opportunity_ID", "Prediction"])
prev["Prediction"].corr(answer["Prediction"])

0.9895329847287113

In [138]:
prev = pd.read_csv("acceptable/submission_11.csv")
prev["Prediction"].corr(answer["Prediction"])

0.9905176422206324

In [140]:
answer["Opportunity_ID"] = pd.

In [141]:
answer.to_csv("acceptable/submission_28.csv", header = False, index = False)